[Website](https://nigeriapropertycentre.com/for-sale/houses/showtype)

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import pandas as pd
import time

In [2]:
# url = "https://nigeriapropertycentre.com/for-sale/houses/showtype?page=1"

url = "https://nigeriapropertycentre.com/for-sale/houses/detached-duplexes/lagos/lekki/ikota/2559353-five-bedroom-fully-detached-duplex"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html')
response

<Response [200]>

In [ ]:
properties = []
page_count = 0
max_pages = 1

path = "C:/Users/HP/Downloads/chromedriver-win64/chromedriver.exe"
url = "https://nigeriapropertycentre.com/for-sale/houses/showtype"

service = Service(path)
driver = webdriver.Chrome(service=service)

driver.get(url)
property_links = [link.get_attribute('href') for link in driver.find_elements(By.XPATH, '//a[@itemprop="url"]')]

while page_count<=max_pages:

    time.sleep(3)

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html')

    for link_url in property_links:
        driver.get(link_url)
        time.sleep(3)

        rows = soup.find_all('table', class_= 'table table-bordered table-striped')        
        
        for row in rows:
            line = row.find_all('td')
            fields = [h.text.strip() for h in line]
            
            table = {
                field.split(":")[0].strip(): field.split(":")[1].strip() 
                for field in fields if ":" in field # and len(field.split(":")) > 1
                }

            figure = soup.find_all('span', class_='price')[1]
            price = float(figure.attrs['content'])
            table['Price'] = price

            address = soup.find('div', class_='col-sm-8 f15 property-details')
            location = address.text.strip().split(',')[-2:]
            city = location[0].strip()
            state = location[1].strip()
            table['District'] = city 
            table['State'] = state

            properties.append(table)

            # back = driver.find_element(By.XPATH, '//a[@class="underline"]')
            # back.click()
        driver.back()
        time.sleep(3) 

    page_count +=1
    print(f'Scraped page {page_count} of {max_pages}.')

    try:
        next_icon = driver.find_element(By.XPATH, "//a[@rel='next']")
        next_icon.click()
    except:
        print("No more pages to scrape")
        break

# driver.quit()

Scraped page 1 of 2.
Scraped page 2 of 2.


In [9]:
properties

[]